In [1]:
# Install libraries
!pip install scikit-learn pandas numpy matplotlib seaborn

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
from google.colab import files
uploaded = files.upload()


Saving house_prices.csv to house_prices.csv


In [3]:
import pandas as pd

df = pd.read_csv(list(uploaded.keys())[0])
df.head()


,Index,Title,Description,Amount(in rupees),Price (in rupees),location,Carpet Area,Status,Floor,Transaction,...,facing,overlooking,Society,Bathroom,Balcony,Car Parking,Ownership,Super Area,Dimensions,Plot Area
0,0,1 BHK Ready to Occupy Flat for sale in Srushti...,"Bhiwandi, Thane has an attractive 1 BHK Flat f...",42 Lac,6000.0,thane,500 sqft,Ready to Move,10 out of 11,Resale,...,NaN,NaN,Srushti Siddhi Mangal Murti Complex,1,2,NaN,NaN,NaN,NaN,NaN
1,1,2 BHK Ready to Occupy Flat for sale in Dosti V...,One can find this stunning 2 BHK flat for sale...,98 Lac,13799.0,thane,473 sqft,Ready to Move,3 out of 22,Resale,...,East,Garden/Park,Dosti Vihar,2,NaN,1 Open,Freehold,NaN,NaN,NaN
2,2,2 BHK Ready to Occupy Flat for sale in Sunrise...,Up for immediate sale is a 2 BHK apartment in ...,1.40 Cr,17500.0,thane,779 sqft,Ready to Move,10 out of 29,Resale,...,East,Garden/Park,Sunrise by Kalpataru,2,NaN,1 Covered,Freehold,NaN,NaN,NaN
3,3,1 BHK Ready to Occupy Flat for sale Kasheli,This beautiful 1 BHK Flat is available for sal...,25 Lac,NaN,thane,530 sqft,Ready to Move,1 out of 3,Resale,...,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN
4,4,2 BHK Ready to Occupy Flat for sale in TenX Ha...,"This lovely 2 BHK Flat in Pokhran Road, Thane ...",1.60 Cr,18824.0,thane,635 sqft,Ready to Move,20 out of 42,Resale,...,West,"Garden/Park, Main Road",TenX Habitat Raymond Realty,2,NaN,1 Covered,Co-operative Society,NaN,NaN,NaN


In [4]:
# Clean “Amount(in rupees)” — convert Lac/Cr to number
def convert_price(value):
    if pd.isna(value):
        return np.nan
    value = value.strip()
    if "Cr" in value:
        return float(value.replace("Cr","").strip()) * 10000000
    if "Lac" in value:
        return float(value.replace("Lac","").strip()) * 100000
    try:
        return float(value)
    except:
        return np.nan

df['Amount(in rupees)'] = df['Amount(in rupees)'].apply(convert_price)


In [5]:
#Convert Carpet Area (“500 sqft”) to number
df['Carpet Area'] = df['Carpet Area'].str.extract('(\d+)').astype(float)


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-4008771034.py:2: SyntaxWarning: invalid escape sequence '\d'
  df['Carpet Area'] = df['Carpet Area'].str.extract('(\d+)').astype(float)


In [6]:
#Convert Floor (“10 out of 11”) into two numerical columns
df[['Floor Number','Total Floors']] = df['Floor'].str.extract(r'(\d+)\s*out of\s*(\d+)?')
df['Floor Number'] = df['Floor Number'].astype(float)
df['Total Floors'] = df['Total Floors'].astype(float)


In [7]:
#Drop useless string columns (Title, Description, Society, etc.)
df = df.drop(['Title','Description','Society','Dimensions','Plot Area'], axis=1)


In [8]:
#Fill missing numeric values
for col in df.select_dtypes(include=['float64','int64']):
    df[col] = df[col].fillna(df[col].median())


In [9]:
#Label encode all remaining categorical columns
from sklearn.preprocessing import LabelEncoder

label_cols = df.select_dtypes(include=['object']).columns

le_dict = {}
for col in label_cols:
    le = LabelEncoder()
    df[col] = df[col].astype(str)
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le


In [10]:
#Build predictive model (Random Forest works best on mixed data)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X = df.drop(['Price (in rupees)'], axis=1)
y = df['Price (in rupees)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R² Score:", r2_score(y_test, y_pred))


RMSE: 12072.983042713238
R² Score: 0.6800684341113472


In [11]:
# Predict price for a new property
sample = X_test.iloc[0:1]
model.predict(sample)


array([9646.])